In [1]:
import pandas as pd

In [2]:
trackings = pd.read_csv("/work/data/input/nfl-health-and-safety-helmet-assignment/train_player_tracking.csv")

In [8]:
# copied from https://www.kaggle.com/robikscube/nfl-helmet-assignment-getting-started-guide
def add_track_features(tracks, fps=59.94, snap_frame=10):
    """
    Add column features helpful for syncing with video data.
    """
    tracks = tracks.copy()
    tracks["game_play"] = (
        tracks["gameKey"].astype("str")
        + "_"
        + tracks["playID"].astype("str").str.zfill(6)
    )
    tracks["time"] = pd.to_datetime(tracks["time"])
    snap_dict = (
        tracks.query('event == "ball_snap"')
        .groupby("game_play")["time"]
        .first()
        .to_dict()
    )
    tracks["snap"] = tracks["game_play"].map(snap_dict)
    tracks["isSnap"] = tracks["snap"] == tracks["time"]
    tracks["team"] = tracks["player"].str[0].replace("H", "Home").replace("V", "Away")
    tracks["snap_offset"] = (tracks["time"] - tracks["snap"]).astype(
        "timedelta64[ms]"
    ) / 1_000
    # Estimated video frame
    tracks["est_frame"] = (
        ((tracks["snap_offset"] * fps) + snap_frame).round().astype("int")
    )
    return tracks
trackings = add_track_features(trackings)

In [13]:
#trackings[trackings["event"] == "ball_snap"]
trackings

,gameKey,playID,player,time,x,y,s,a,dis,o,dir,event,game_play,snap,isSnap,team,snap_offset,est_frame
0,57583,82,H97,2018-09-14 00:23:45.500000+00:00,38.89,28.06,0.00,0.00,0.00,261.28,222.79,NaN,57583_000082,2018-09-14 00:24:00.200000+00:00,False,Home,-14.700,-871
1,57583,82,H97,2018-09-14 00:23:45.599000+00:00,38.89,28.06,0.00,0.00,0.00,261.28,221.93,NaN,57583_000082,2018-09-14 00:24:00.200000+00:00,False,Home,-14.601,-865
2,57583,82,H97,2018-09-14 00:23:45.700000+00:00,38.89,28.06,0.00,0.00,0.00,261.28,221.21,NaN,57583_000082,2018-09-14 00:24:00.200000+00:00,False,Home,-14.500,-859
3,57583,82,H97,2018-09-14 00:23:45.799000+00:00,38.88,28.05,0.00,0.00,0.00,261.28,220.07,NaN,57583_000082,2018-09-14 00:24:00.200000+00:00,False,Home,-14.401,-853
4,57583,82,H97,2018-09-14 00:23:45.900000+00:00,38.88,28.05,0.00,0.00,0.00,261.28,220.31,NaN,57583_000082,2018-09-14 00:24:00.200000+00:00,False,Home,-14.300,-847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333806,58107,4362,V53,2019-12-10 04:33:24.099000+00:00,105.15,36.08,0.38,0.35,0.04,279.23,330.97,NaN,58107_004362,2019-12-10 04:33:10.799000+00:00,False,Away,13.300,807
333807,58107,4362,V53,2019-12-10 04:33:24.200000+00:00,105.13,36.09,0.29,0.85,0.03,273.65,300.97,NaN,58107_004362,2019-12-10 04:33:10.799000+00:00,False,Away,13.401,813
333808,58107,4362,V53,2019-12-10 04:33:24.299000+00:00,105.11,36.10,0.23,0.91,0.02,268.49,279.58,NaN,58107_004362,2019-12-10 04:33:10.799000+00:00,False,Away,13.500,819
333809,58107,4362,V53,2019-12-10 04:33:24.400000+00:00,105.08,36.11,0.21,0.69,0.02,266.05,277.68,NaN,58107_004362,2019-12-10 04:33:10.799000+00:00,False,Away,13.601,825
